In [16]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

In [17]:
# build the VGG16 network
input_tensor = Input(shape=(224,224,3))
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

In [18]:
# build a classifier model to put on top of the convolutional model
x = model.output
x = Flatten(input_shape=(model.output_shape[1:]))(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dense(4, activation='softmax', name='output', kernel_initializer='glorot_uniform')(x)

# add new classifier model on top of convolutional base
new_model = Model(model.input, x)

In [19]:
# set the first 19 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in new_model.layers[:19]:
    layer.trainable = False

In [20]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [21]:
# Use SGD and Categorical CE Loss
#sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# RMSprop
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

In [22]:
# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("../datasets/bikes_classified.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        label = row[3]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(str(label))

In [23]:

def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 4)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 4)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)
                # Convert to 1 hot vector
                p = prices[index]
                if p == "25":
                    Y[i,:] = np.array([1,0,0,0])
                if p == "50":
                    Y[i,:] = np.array([0,1,0,0])
                if p == "75":
                    Y[i,:] = np.array([0,0,1,0])
                if p == "100":
                    Y[i,:] = np.array([0,0,0,1])
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [24]:
train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

(19658,)
(2185,)


In [25]:
from keras.callbacks import ModelCheckpoint

num_epochs = 100
minibatch_size = 64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

checkpoint_path = 'E:/output/bikes-cnn-VGG16-Class/{epoch:05d}.hdf5'
checkpoint = ModelCheckpoint(checkpoint_path, period=5)

# fine-tune the model
history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=25,
    validation_data=image_generator(test_indices, minibatch_size),
    nb_val_samples=test_steps,
    callbacks=[checkpoint])

C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=35, validation_data=<generator..., epochs=25, steps_per_epoch=35, callbacks=[<keras.ca...)`


Epoch 1/25
35/35 [==============================] - 19s 557ms/step - loss: 8.1949 - acc: 0.4312 - val_loss: 6.5812 - val_acc: 0.5104
Epoch 2/25
35/35 [==============================] - 19s 529ms/step - loss: 6.1901 - acc: 0.5425 - val_loss: 4.3833 - val_acc: 0.6381
Epoch 3/25
35/35 [==============================] - 19s 530ms/step - loss: 5.0377 - acc: 0.6056 - val_loss: 6.7617 - val_acc: 0.5064
Epoch 4/25
35/35 [==============================] - 19s 552ms/step - loss: 4.6777 - acc: 0.6332 - val_loss: 3.2073 - val_acc: 0.7190
Epoch 5/25
35/35 [==============================] - 19s 550ms/step - loss: 3.9990 - acc: 0.6748 - val_loss: 2.6162 - val_acc: 0.7799
Epoch 6/25
35/35 [==============================] - 19s 557ms/step - loss: 3.6129 - acc: 0.7023 - val_loss: 2.7795 - val_acc: 0.7550
Epoch 7/25
35/35 [==============================] - 19s 550ms/step - loss: 3.2043 - acc: 0.7214 - val_loss: 3.0932 - val_acc: 0.7359
Epoch 8/25
35/35 [==============================] - 19s 551ms/step - 

In [6]:
from keras.models import load_model
new_model = load_model('../datasets/bikes_classification_best.hdf5')

In [26]:
true_label = []
predicted_label = []
for index in test_indices:
    msrp = prices[index]
    true_label.append(str(msrp))
    
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    
    # Prediction outputs softmax vector
    prediction = new_model.predict(data)
    
    # Set most confident prediction as label, and convert it to our price scale
    label = np.argmax(prediction) * 25 + 25
    predicted_label.append(str(label))

In [27]:
from sklearn.metrics import confusion_matrix, classification_report
print("Classification report:\n%s\n"
      % (classification_report(true_label, predicted_label)))
print("Confusion matrix:\n%s" % confusion_matrix(true_label, predicted_label))

Classification report:
             precision    recall  f1-score   support

        100       0.79      0.99      0.88       496
         25       0.92      0.97      0.95       596
         50       0.95      0.82      0.88       563
         75       0.90      0.75      0.82       530

avg / total       0.89      0.88      0.88      2185


Confusion matrix:
[[490   0   1   5]
 [  2 578  12   4]
 [ 23  39 464  37]
 [109   9  12 400]]


In [ ]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

# build the VGG16 network
input_tensor = Input(shape=(224,224,3))
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

# build a classifier model to put on top of the convolutional model
x = model.output
x = Flatten(input_shape=(model.output_shape[1:]))(x)
x = Dropout(0.2)(x)
x = Dense(300, activation='relu', kernel_initializer='glorot_normal')(x)
x = Dropout(0.2)(x)
x = Dense(300, activation='relu', kernel_initializer='glorot_normal')(x)
x = Dense(5, activation='softmax', name='output', kernel_initializer='glorot_normal')(x)

# add new classifier model on top of convolutional base
new_model = Model(model.input, x)

# set the first 19 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in new_model.layers[:19]:
    layer.trainable = False
    
# Use SGD and Categorical CE Loss
#sgd = optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#new_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(lr=0.0003), metrics=['accuracy'])

# RMSprop
# new_model.compile(loss='categorical_crossentropy',
#                   optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])


# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/cars_im/"
with open("../datasets/cars_classified.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[3]
        label = row[4]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(str(label))
        
        
def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 5)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 5)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)
                # Convert to 1 hot vector
                p = prices[index]
                if p == "20":
                    Y[i,:] = np.array([1,0,0,0,0])
                if p == "40":
                    Y[i,:] = np.array([0,1,0,0,0])
                if p == "60":
                    Y[i,:] = np.array([0,0,1,0,0])
                if p == "80":
                    Y[i,:] = np.array([0,0,0,1,0])
                if p == "100":
                    Y[i,:] = np.array([0,0,0,0,1])
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)
            
train_indices = np.load("cars_train_indices.npy")
test_indices = np.load("cars_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

epochs = 10
minibatch_size = 64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

checkpoint = ModelCheckpoint('E:/output/cars-cnn-VGG16-Class/{epoch:05d}.hdf5', period=5)

# fine-tune the model
history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs,
    validation_data=image_generator(test_indices, minibatch_size),
    nb_val_samples=test_steps,
    callbacks=[checkpoint])
# new_model.save('cars_classification_best.hdf5')


true_label = []
predicted_label = []
for index in test_indices:
    msrp = prices[index]
    true_label.append(str(msrp))
    
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    data = preprocess_input(data)
    
    # Prediction outputs softmax vector
    prediction = new_model.predict(data)
    
    # Set most confident prediction as label, and convert it to our price scale
    label = np.argmax(prediction) * 20 + 20
    predicted_label.append(str(label))
    
from sklearn.metrics import confusion_matrix, classification_report
print("Classification report:\n%s\n"
      % (classification_report(true_label, predicted_label)))
print("Confusion matrix:\n%s" % confusion_matrix(true_label, predicted_label))

(1330,)
(147,)


C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:134: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:134: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., callbacks=[<keras.ca..., epochs=10, validation_steps=3, validation_data=<generator..., steps_per_epoch=3)`


Epoch 1/10
3/3 [==============================] - 3s 1s/step - loss: 8.5170 - acc: 0.3146 - val_loss: 5.5877 - val_acc: 0.4787
Epoch 2/10
3/3 [==============================] - 2s 649ms/step - loss: 7.6064 - acc: 0.3767 - val_loss: 7.4784 - val_acc: 0.4783
Epoch 3/10
3/3 [==============================] - 2s 599ms/step - loss: 6.6348 - acc: 0.5143 - val_loss: 4.5015 - val_acc: 0.6066
Epoch 4/10
1/3 [=========>....................] - ETA: 0s - loss: 4.7735 - acc: 0.5783